# finding model noise
This code is extracted from `Enkf.ipynb` and is written to run only once and to save the Error matrix and Covariance matrix, so for the future uses I dont find the error every time.


$dt$ is a very important constant in this piece of code, it changes the noise.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch
import sys
sys.path.append('./..')
import cte
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from ProcessFunctions import find_Aisv2,Find_a_i,find_Aisv2_onlyv,Find_T_X_tau_without_p_input
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch.nn.functional as F
from scipy.interpolate import interp1d
import seaborn as sns 
from scipy import integrate
import scipy
import cte_eq

# Loading POD modes
T_final=10500
Ntout=cte_eq.Ntout
Nxout=cte_eq.Nxout
drs=0.012
mu=3e10

data_dir='/central/groups/Avouac_lab/hkaveh/Data/LearnROM/2D/MainSimulation2D_Tf'+str(T_final)+"Nt="+str(Ntout)+"Nx="+str(Nxout)+'PODonlyonV'+'drs'+str(drs)+".npz"
PODmodes=np.load(data_dir)
U_v=PODmodes['U']
S_v=PODmodes['S']
# VT_v=PODmodes['VT']
q_bar_v=PODmodes['q_bar']
Sigma_v=np.diagonal(S_v)
Nt2=127534 # This is the number os snapshots used to find the POD, it is fined in the RunForward2D.ipynb
Lambda_v=Sigma_v**2/Nt2 # Covariance matrix


data_dir='/central/groups/Avouac_lab/hkaveh/Data/LearnROM/2D/MainSimulation2D_Tf'+str(T_final)+"Nt="+str(Ntout)+"Nx="+str(Nxout)+'PODonlyontheta'+'drs'+str(drs)+".npz"
PODmodes=np.load(data_dir)
U_theta=PODmodes['U']
S_theta=PODmodes['S']
# VT_theta=PODmodes['VT']
q_bar_theta=PODmodes['q_bar']
Sigma_theta=np.diagonal(S_theta)
# Nt2=VT_theta.shape[0]
Lambda_theta=Sigma_theta**2/Nt2 # Covariance matrix
N_m_load=30
phi=U_v[:,:N_m_load]    # This contains the eigen mode for velocity


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
dt=3600*6          #                for SSEs I used             5*24*3600 
gamma_ratio=0.0001*5 # variance of the observation noise is multiplied by the value
sigma_kernel=2000 # in meters



N_m_v_list=[20,25,30]
N_m_theta_list=[20,25,30]
index=0
N_m_theta=N_m_theta_list[index]
N_m_v=N_m_v_list[index]

Lambda=np.append(Lambda_v[:N_m_v],Lambda_theta[:N_m_theta]) # Lambda is the covariance matrix
Lambda=np.diag(Lambda)

Gamma=np.diag(Lambda_v[:N_m_v])

t_yr=365*24*3600           # number of seconds in a year
Nx=1024
L_thresh=1e3
L=240e3
L_fault=L
x_ox=np.linspace(-L/2,L/2,Nx)
L_element_x=L/Nx

sigma_x=sigma_kernel/L_element_x # in number of elements

Obs_linear=0
inflation=1  # inflection of the model noise covariance matrix

In [ ]:
print(L_element_x)
print(sigma_x)
print("for SSE example sigma_x was 1.6")

# Noise
I start with noise because it is the easiest. What do you need to make noise? probably the standard deviation of the POD modes. Some assumption on the ratio of the noise. Ok, how can you get the standard deviation? I think I can just import it.

In [ ]:
# defining the model noise as xi \in R^n which is a normal distribution with mean 0 and std proportional to Lambda,
# We take the covarinace matrix of the model noise (remember covariance as the unit standard deviation squared)
# Here covariance matrix is Sigma

# def xi_call2(Lambda,xi_ratio,m):
#     xi=np.random.multivariate_normal(np.zeros(Lambda.shape[0]),xi_ratio*Lambda,m)
#     return xi.T

def xi_call(N_m,cov,m):
    xi=np.random.multivariate_normal(np.zeros(N_m),cov,m)
    return xi.T

In [ ]:
# defining the observation noise as eta \in R^d which is a normal distribution with mean 0 and std proportional to Lambda_y,
def eta_call(Gamma,gamma_ratio,m):
    eta=np.random.multivariate_normal(np.zeros(Gamma.shape[0]),gamma_ratio*Gamma,m)
    return eta.T

In [ ]:
# import the model from the code MLmodel8postprocess.ipynb
# %% Defining model (g_1)
class Forwardmap(nn.Module):
    def __init__(self, N_m, dropout_rate=0.2, negative_slope=0.01):
        super().__init__()
        self.hidden1 = nn.Linear(N_m, 2 * N_m)
        self.act1 = nn.LeakyReLU(negative_slope)
        self.hidden2 = nn.Linear(2 * N_m, 4 * N_m)
        self.act2 = nn.LeakyReLU(negative_slope)
        self.hidden3 = nn.Linear(4 * N_m, 4 * N_m)
        self.act3 = nn.LeakyReLU(negative_slope)
        self.hidden4 = nn.Linear(4 * N_m, 2 * N_m)
        self.act4 = nn.LeakyReLU(negative_slope)
        self.output = nn.Linear(2 * N_m, N_m)
        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.dropout(x)
        x = self.act2(self.hidden2(x))
        x = self.dropout(x)
        x = self.act3(self.hidden3(x))
        x = self.dropout(x)
        x = self.act4(self.hidden4(x))
        x = self.dropout(x)
        x = self.output(x)
        return x
version=0
    
model=Forwardmap(N_m_v+N_m_theta)
model.load_state_dict(torch.load(
    "/central/groups/astuart/hkaveh/Data/LearnROM/Earthquake2D_ROM_POD_separate_onV_theta"
    + str(version)
    + "N_m_v"
    + str(N_m_v)
    + "N_m_theta"
    + str(N_m_theta)
    + ".pt",
    map_location=torch.device('cpu')
))
model.eval()
model=model.to(device)

In [ ]:
# The reduced order model for v is already loaded
# lets load the model for dt
# Loading the pytorch ML model
version_dt=0
class Forwardmapdt(nn.Module):
    def __init__(self,N_m_v, dropout_rate=0.2, negative_slope=0.01):
        super().__init__()
        self.hidden1=nn.Linear(N_m_v+1,2*N_m_v)
        self.act1=nn.LeakyReLU(negative_slope)
        self.hidden2=nn.Linear(2*N_m_v,4*N_m_v)
        self.act2=nn.LeakyReLU(negative_slope)
        self.hidden3=nn.Linear(4*N_m_v,4*N_m_v)
        self.act3=nn.LeakyReLU(negative_slope)
        self.hidden4=nn.Linear(4*N_m_v,2*N_m_v)
        self.act4=nn.LeakyReLU(negative_slope)
        self.output=nn.Linear(2*N_m_v,1)
        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self,x):
        x=self.act1(self.hidden1(x))
        x = self.dropout(x)
        x=self.act2(self.hidden2(x))
        x = self.dropout(x)
        x=self.act3(self.hidden3(x))
        x = self.dropout(x)
        x=self.act4(self.hidden4(x))
        x = self.dropout(x)
        x=(self.output(x))
        return x
    
modeldt=Forwardmapdt(N_m_v)
modeldt.load_state_dict( torch.load("/central/groups/Avouac_lab/hkaveh/Data/LearnROM/2D/EQ_ROM_g2"+str(version)+"N_m_v"+str(N_m_v)+".pt"))
modeldt.eval()
modeldt=modeldt.to(device)


In [ ]:

phi_torch     = torch.tensor(phi[:,:20], dtype=torch.float32, device=device)
q_bar_v_torch = torch.tensor(q_bar_v, dtype=torch.float32, device=device)

def find_logvmax(alpha):
    if not isinstance(alpha, torch.Tensor):
        alpha = torch.tensor(alpha, dtype=phi_torch.dtype, device=phi_torch.device)
    v = torch.matmul(phi_torch, alpha).unsqueeze(1) + q_bar_v_torch
    logvmax = torch.max(v).detach()  # Get the maximum value in v
    return logvmax

In [ ]:
params_f=[N_m_v+N_m_theta]
rho=2
lambda_dissipation=0.1

In [ ]:
def dissipation_factor(xk,rho,Sigma,Beta=20):
    # u=xk/Sigma # normalize by the std
    alpha=xk[:,:N_m_v]
    logvmax=find_logvmax(alpha.T)
    # norm_u=torch.norm(u)
    output=1/(1+torch.exp(Beta*(logvmax-rho)))
    return output
directory='/central/groups/Avouac_lab/hkaveh/Data/LearnROM/2D/sigma'+str(T_final)+"Nt="+str(Ntout)+"Nx="+str(Nxout)+'PODonlyonV'+'drs'+str(drs)+".npz"
Sigma_v=np.load(directory)['Sigma_v']
Sigma_theta=np.load(directory)['Sigma_theta']
Sigma=np.concatenate((Sigma_v,Sigma_theta))
Sigma = torch.tensor(Sigma, dtype=torch.float32, device=device)

In [ ]:
# This function is written to find the next time step in the Kalman filter algorithm and is the forward model
# This function needs to be double checked.
def f(alpha_k,dt,params_f):
    # alpha_k is the state at time k
    # alpha_kp1 is the state at time k+1
    # dt is the time step (assume in seconds)
    N_m=params_f[0]
    xk=torch.tensor(alpha_k, dtype=torch.float32)           # xk is in torch.
    forecast=xk.detach().cpu().numpy().reshape(N_m,1)       # forecast is in cpu.
    time=np.array([0])                                      # time is in cpu
    while time[-1]<dt:
        xk=xk.to(device,dtype=torch.float32)
        dissipate=dissipation_factor(xk.reshape((1,N_m)),rho,Sigma)          # torch
        ykp1=(model(xk))                                    # ykp1 is the prediction of the model in torch
        xkp1=(ykp1+xk)* dissipate + (1-dissipate)*(lambda_dissipation*xk) # This is the prediction of the model in torch

        numpy_vector = xkp1.detach().cpu().numpy().reshape(N_m,1)
        x_ML=numpy_vector                             # This is the alpha that has both v and theta
        vmax=find_logvmax(x_ML[:N_m_v])       # This is the maximum of v

        x_input_dt_ML=x_ML[:N_m_v]                 # This is the input for the dt model (except vmax)
        x_input_dt_ML=np.append(x_input_dt_ML,vmax.detach().cpu().numpy())         # This is the input for the dt model
        x_input_dt_ML=torch.tensor(x_input_dt_ML, dtype=torch.float32) # This is the input for the dt model
        x_input_dt_ML=x_input_dt_ML.to(device)
        dt_pred=modeldt(x_input_dt_ML)                      # This is the prediction for the time step
        dt_pred=10 ** dt_pred.detach().cpu().double().numpy().astype(np.float64)    # This is the prediction for the time step
        time=np.append(time,dt_pred+time[-1])                        # This is the time vector
        forecast = np.hstack([forecast, numpy_vector])      # This is the forecasted alpha (not scaled though)
        xk=xkp1
    
    # applying linear interpolation to find forecast at time dt, using only the last two points
    f2 = interp1d([time[-2],time[-1]], forecast[:,-2:],axis=1)
    alpha_kp1=f2(dt).reshape(N_m,1)
    # Removing the last coulmn of forecast becasue it is for the time when t > dt
    forecast=forecast[:,:-1]
    time[-1]=dt
    # Then appending it to the forecast
    forecast = np.concatenate((forecast, alpha_kp1),axis=1)
    return forecast,time


I did it untill here.

# Uniform time stepping discrete map

To do:

* Import and load the model snapshot and timestep.
* pick an initial condition. randomly. I tested the code with a few initial conditiions and checked the invariant measure and they look similar to the invariant measure of the system.
* This is what I am going to do. In each iteration, I find the forecast, then multiply it by max_X to get alpha without scales. Then I find the vmax from the unscaled data. Then I divide alpha by `max_X_dt` and concatinate with vmax to have the input for the machine learning model of dt.  Then I run it through the ML model for time stepping and then to get the actual dt, I need to multiply the output by max_Y_dt.
* Write the thing in a function. in the form: $\alpha(t_{n+1})=f(\alpha(t_n),t_n)$

In [ ]:
# # This function is written to find the next time step in the Kalman filter algorithm and is the forward model
# def f(alpha_k,dt,params_f):
#     # alpha_k is the state at time k
#     # alpha_kp1 is the state at time k+1
#     # dt is the time step (assume in seconds)
#     max_X=params_f[0]
#     max_y=params_f[1]
#     max_X_dt=params_f[2]
#     max_Y_dt=params_f[3]
#     N_m_v=params_f[4]
#     phi=params_f[5]
#     N_m=params_f[6]
#     xk=torch.tensor(alpha_k/max_X, dtype=torch.float32)
#     forecast=xk.detach().numpy().reshape(N_m,1)
#     time=np.array([0])
#     while time[-1]<dt:
#         xk.to(device)
#         ykp1=(model(xk))*(max_y/max_X)
#         xkp1=ykp1+xk
#         xk=xkp1
#         numpy_vector = xkp1.detach().numpy().reshape(N_m,1)
#         x_ML=numpy_vector*max_X                             # This is the alpha that has both v and theta
#         vmax=find_logvmax(x_ML[:N_m_v],phi[:,:N_m_v])       # This is the maximum of v
#         x_input_dt_ML=x_ML[:N_m_v]/max_X_dt                 # This is the input for the dt model (except vmax)
#         x_input_dt_ML=np.append(x_input_dt_ML,vmax)         # This is the input for the dt model
#         x_input_dt_ML=torch.tensor(x_input_dt_ML, dtype=torch.float32) # This is the input for the dt model
#         x_input_dt_ML.to(device)
#         dt_pred=modeldt(x_input_dt_ML)                      # This is the prediction for the time step
#         dt_pred=10**(dt_pred.detach().numpy()*max_Y_dt)     # This is the prediction for the time step
#         time=np.append(time,dt_pred+time[-1])                        # This is the time vector
#         forecast = np.hstack([forecast, numpy_vector])      # This is the forecasted alpha (not scaled though)
#     # applying linear interpolation to find forecast at time dt, using only the last two points
#     f = interp1d([time[-2],time[-1]], forecast[:,-2:],axis=1)
#     alpha_kp1=f(dt).reshape(N_m,1)
#     # Removing the last coulmn of forecast becasue it is for the time when t > dt
#     forecast=forecast[:,:-1]
#     time[-1]=dt
#     # Then appending it to the forecast
#     forecast = np.concatenate((forecast, alpha_kp1),axis=1)*max_X
#     return forecast,time


## Checking the forwawrd model

## Running the forward model for a long time (to check the statistics):
To plot (the scaling and invariant measure) you can uncomment the following cells.

In [ ]:
# # This cell runs the program for up to time dt
# dt=100*24*3600*365
# size_x=256
# size_y=32
# size=U_v.shape[0]
# alpha_k=xi_call(40,Lambda,1).T
# forecast,time=f(alpha_k,dt,params_f)

In [ ]:
# # This cell calculate the things we need for finding the scaling laws.
# vmaxs=[]
# V_ox_ROM=np.empty((forecast.shape[1],size_y,size_x))
# t_ox_ROM=np.ones((forecast.shape[1],size_y,size_x))
# U_ROM=U_v[:,0:N_m_v]

# for j in range(forecast.shape[1]):
#     V_snapshot = U_ROM @ (forecast[:N_m_v, j]) + q_bar_v.reshape(size,)
#     V_snapshot = V_snapshot[:size_x * size_y].reshape(size_y, size_x)
#     V_ox_ROM[j,:,:]=10**(V_snapshot)
#     t_ox_ROM[j,:,:]=time[j]*np.ones((size_y,size_x))
#     vmaxs.append(np.max(V_snapshot))
# TimeStarts_ML,TimeEnds_ML,rectangles_ML,Mags_ML=Find_T_X_tau_without_p_input(V_ox_ROM,t_ox_ROM,V_thresh,L_thresh,t_yr,x_ox,z_ox,L_fault,mu)
 

## Loading QDYN simulation and scalings

In [ ]:
# def FindMwupdated(V_full,t_full,V_thresh):
#     flag=0
#     V_max=np.max(V_full,axis=(1,2))
#     Mw=np.array([])
#     T1=np.array([]) # it is the time of when the earthquakes nucleate
#     T2=np.array([]) # it is the time of when the earthquake stops
#     PotRate=np.sum(V_full,axis=(1,2))*Area/Nx/Nz

#     for i in range(V_max.size):
        
#         if flag==0 and V_max[i]>V_thresh: # an event has started
#             flag=1
#             index1=i
#             T1=np.append(T1,t_full[i])
#         if flag==1 and V_max[i]<V_thresh: # the event has stopped
#             flag=0
#             index2=i
#             IntPotRate=integrate.cumtrapz(PotRate[index1:index2+1],t_full[index1:index2+1])
#             Integration=IntPotRate[-1]
#             M0=Integration*mu
#             Mw=np.append(Mw,(2/3)*np.log10(M0)-6)
#             T2=np.append(T2,t_full[i])
#     return Mw,T1,T2

In [ ]:
# # I need to load some of the already run models to check the scaling, after checking you can commentout this cell
# # Loading the data from the simulation
# # Loading data from "/central/groups/astuart/hkaveh/Data/LearnROM/"
# Tf = 250 # each initial condition is simulated for 250 years
# Nt=5     # it is recorded every Nt time stes

# N_m_load=30
# coeff=2 # How the inital conditions are from the chaotic attractor
# N_cut=2000
# filter_ratio=0.4 # removing 40 percent of the data
# # loading time series:


# Mws_v2=np.array([])
# T1s_v2=np.array([])
# T2s_v2=np.array([])
# rectangles_all=np.array([])

# findscaling=1
# X_full=np.empty((1,N_m_load*2))

# for number in range(10):
#     data_dir="/central/groups/astuart/hkaveh/Data/LearnROM/SampleSimulation_Tf"+str(Tf)+"Nt="+str(Nt)+"N_m"+str(N_m_load)+"coeff"+str(coeff)+"number"+str(number)+".npz"
#     data_smaple=np.load(data_dir)
#     V_ox=data_smaple['array1']
#     theta_ox=data_smaple['array2']
#     t_ox=data_smaple['array3']
#     # We work we log10 of V_ox:
#     t=t_ox[:,0,0].reshape(-1,1)
#     Start_index=int(V_ox.shape[0]*filter_ratio)
    

#     if findscaling==1:

#         TimeStarts,TimeEnds,rectangles,Mags=Find_T_X_tau_without_p_input(V_ox,t_ox,V_thresh,L_thresh,t_yr,x_ox,z_ox,L_fault,mu)
#         Mws_v2=np.append(Mws_v2,Mags)
#         T1s_v2=np.append(T1s_v2,TimeStarts)
#         T2s_v2=np.append(T2s_v2,TimeEnds)
#         rectangles_all=np.append(rectangles_all,rectangles)
#     V_ox=np.log10(V_ox)
#     theta_ox=np.log10(theta_ox)
#     A_v,P_v=find_Aisv2_onlyv(U_v,V_ox,q_bar_v,N_m_load) # you dont need P in general, but I need it to check if I do everything correctly
#     A_theta,P_theta=find_Aisv2_onlyv(U_theta,theta_ox,q_bar_theta,N_m_load)
#     A=np.concatenate((A_v,A_theta),axis=1)
#     X_onesimulation=A[Start_index:-1,:] # with removig the first 20 percent of the data to remove the transient
#     X_full=np.append(X_full,X_onesimulation,axis=0)
# X_full=X_full[1:,:]

# Nrectangles=int(rectangles_all.size/4)
# rectangles_all=np.reshape(rectangles_all,(Nrectangles,4))
# Areas=W*(rectangles_all[:,3])*1000

In [ ]:
# # Checking the invariant measure in the pytorch model

# fig, axs = plt.subplots(3, 5, figsize=(15, 10))

# # Flatten the axs array so that we can iterate over it easily
# axs = axs.flatten()

# # Plot the KDE plot for each column of X_full
# for i in range(15):
#     sns.kdeplot(forecast[i,:], ax=axs[i],label='ROM (n=40)')
#     sns.kdeplot(X_full[:, i], ax=axs[i],label='QDYN')
#     axs[i].set_title(r'$\alpha_{{{}}}$'.format(i+1))
#     axs[i].legend()


# # Adjust layout and show plot
# plt.tight_layout()
# plt.show()

In [ ]:
# # Plotting the scaling laws:
# plt.rcParams.update({
#     'font.family': 'serif',
#     'font.serif': ['Times New Roman'],
#     'text.usetex': True,  # If you want to use LaTeX for rendering text
# })
# fig, axs = plt.subplots(1, 2, figsize=(8, 3))

# plt.rcParams.update({'font.family': 'serif', 'font.serif': 'Times New Roman','font.size': 8})
# string=str(N_m)
# axs[0].plot(10**(1.5*(Mags_ML+6)),TimeEnds_ML-TimeStarts_ML,'o',label='ROM with n='+str(string),alpha=0.2)
# Nrectangles_ML=int(rectangles_ML.size/4)
# rectangles_ML=np.reshape(rectangles_ML,(Nrectangles_ML,4))
# Areas_ML=W*(rectangles_ML[:,3])*1000
# axs[1].plot(10**(1.5*(Mags_ML+6)),Areas_ML,'o',label='ROM with n='+str(string),alpha=0.2)
# axs[0].plot(10**(1.5*(Mws_v2+6)),T2s_v2-T1s_v2,'o',label='QDYN',color='black',alpha=0.005)  
# axs[1].plot(10**(1.5*(Mws_v2+6)),Areas,'o',label='QDYN',color='black',alpha=0.005)

# axs[0].set_xscale('log')
# axs[0].set_yscale('log')
# axs[0].legend()
# axs[0].set_title('Moment Duration Scaling Law')
# axs[0].set_xlabel(r"$\int_{t_1}^{t_2} \mu \dot{P} dt$")
# axs[0].set_ylabel(r"Duration($s$)")

# axs[1].set_xscale('log')
# axs[1].set_yscale('log')
# axs[1].legend()
# axs[1].set_title('Moment Area Scaling Law')
# axs[1].set_xlabel(r"$\int_{t_1}^{t_2} \mu \dot{P} dt$")
# axs[1].set_ylabel(r"Area ($m^2$)")
# # plot a line with slope 1/3 in log log scale
# x = np.linspace(10**(16),10**18,100)
# y = 10**(1/3*np.log10(x*1e5)) 
# y2= 10**(2/3*np.log10(x*1e-3))
# axs[0].plot(x,y,color='black')
# axs[1].plot(x,y2,color='black')
# for ax in axs:

#     leg = ax.legend()
#     # Set legend handle alpha to 1 for each legend
#     for lh in leg.legendHandles:
#         lh.set_alpha(1)

In [ ]:
# now that I have checked everything, I can wrte function f for uniform time stepping


### Checking the interpolation:
we start from an initial conditon, simulate it for 7 days and then simulate with the same initial conditions for 1 day, 2 days, 3 days 4 days and 5 days, then I plot how does the system look.

In [ ]:
# forecast1,time1=f(alpha_k,1*24*3600,params_f)
# forecast2,time2=f(alpha_k,2*24*3600,params_f)
# forecast3,time3=f(alpha_k,3*24*3600,params_f)
# forecast4,time4=f(alpha_k,4*24*3600,params_f)
# forecast5,time5=f(alpha_k,5*24*3600,params_f)
# forecast6,time6=f(alpha_k,6*24*3600,params_f)




In [ ]:
# plt.plot(time1/24/3600,forecast1[0,:],color='r',linewidth=10)    
# plt.plot(time2/24/3600,forecast2[0,:],color='b',linewidth=8)
# plt.plot(time3/24/3600,forecast3[0,:],color='g',linewidth=6)
# plt.plot(time4/24/3600,forecast4[0,:],color='y',linewidth=4)
# plt.plot(time5/24/3600,forecast5[0,:],color='k',linewidth=2)
# plt.plot(time6/24/3600,forecast6[0,:],color='m',linewidth=1)


Another double check:


In [ ]:
dt/3600/24

In [ ]:
800/60

# Estimating model noise
To find the model noise, I need `X_uniform`, `time_uniform` from the PDE then I need a function that gets `X_uniform(k)` and spits out X_uniform(k+1). I need to find this in a loop and try to find the error that comes from it.

In [ ]:
# step 1: finding X_uniform and time_uniform
# I need to load some of the already run models to check the scaling, after checking you can commentout this cell
# Loading the data from the simulation
# Loading data from "/central/groups/astuart/hkaveh/Data/LearnROM/"
T_final_run = 350 # each initial condition is simulated for 250 years
Nt=5     # it is recorded every Nt time stes
N_m_v=20 # number of modes that they have considered
N_m_theta=20 # number of modes that they have considered
N_m_load=30
coeff=1 # How the inital conditions are from the chaotic attractor
N_cut=2000
filter_ratio=0.5 # I think I should not remove anything, because the error should include data points outside the attractor as well because the particles will end up outside the attractor at some point.
Errors=np.empty((1,N_m_v+N_m_theta))

# loading time series:
for number in range(10):
    data_dir='/central/groups/Avouac_lab/hkaveh/Data/LearnROM/2D/SampleSimulation_Tf_2D'+str(T_final_run)+"Nt="+str(cte_eq.Ntout)+"N_m"+str(N_m_load)+"coeff"+str(coeff)+"number"+str(number)+".npz"
    data_smaple=np.load(data_dir)    
    V_ox=data_smaple['array1']
    theta_ox=data_smaple['array2']
    t_ox=data_smaple['array3']
    t=t_ox[:,0].reshape(-1,1)
    Start_index=int(V_ox.shape[0]*filter_ratio)
    V_ox=np.log10(V_ox)
    theta_ox=np.log10(theta_ox)
    A_v,P_v=find_Aisv2_onlyv(U_v,V_ox,q_bar_v,N_m_load) # you dont need P in general, but I need it to check if I do everything correctly
    A_theta,P_theta=find_Aisv2_onlyv(U_theta,theta_ox,q_bar_theta,N_m_load)
    A=np.concatenate((A_v,A_theta),axis=1)
    X_onesimulation=A[Start_index:-1,:] # with removig the first Start_index data points
    time_nonuniform=t[Start_index:-1,:]
    print("shape X_onesimulation",X_onesimulation.shape)
    print("shape X_onesimulation",time_nonuniform.shape)
    t_0=time_nonuniform[0]
    t_end=time_nonuniform[-1]
    days=int((t_end-t_0)/24/3600)
    index_delete=np.array([])
    if N_m_v<N_m_load:
        index_delete=np.append(index_delete,np.arange(N_m_v,N_m_load))
    if N_m_theta<N_m_load:
        index_delete=np.append(index_delete,np.arange(N_m_load+N_m_theta,2*N_m_load))

    # # Convert the array elements to integers
    index_delete = index_delete.astype(int)

    X_onesimulation=np.delete(X_onesimulation,index_delete,axis=1)
    print(X_onesimulation.shape)
    print(time_nonuniform.shape)
    #make uniform time steps strating from t_0
    N_data=int((t_end-t_0)/dt)-1
    time_uniform=np.linspace(t_0,t_0+N_data*dt,N_data)
    X_uniform=np.empty((N_data,X_onesimulation.shape[1]))
    for i in range(X_onesimulation.shape[1]):
        X_uniform[:,i]=np.interp(time_uniform,time_nonuniform[:,0],X_onesimulation[:,i]).reshape(-1)

    # writing the loop:
    # I need a function (f) that gets `X_uniform(k)` and spits out X_uniform(k+1). I need to find this in a loop and try to find the error that comes from it.
    
    forecast=np.zeros((N_data,X_onesimulation.shape[1]))
    for i in range(N_data):
        forecast[i,:] =f(X_uniform[i,:],dt,params_f)[0][:,-1]
    error=forecast[:-1,:]-X_uniform[1:,:]
    plt.figure()
    plt.plot(X_uniform[1:,0],color='red')
    plt.plot(forecast[:-1,0],color='blue')
    
    Errors=np.append(Errors,error,axis=0)

# # removing the first row of Errors
Errors=Errors[1:,:]
plt.plot(Errors[:,0])


In [ ]:
plt.plot(forecast[:,0],linewidth=5)

plt.plot(X_uniform[:,0])


In [ ]:
# plt.plot(Errors[:,-1])

In [ ]:
Errors.shape

In [ ]:
# saving Errors in Data
np.savez("/central/groups/Avouac_lab/hkaveh/Data/LearnROM/2D/ML_EQ_Errors_N_m_v"+str(N_m_v)+"N_m_theta"+str(N_m_theta)+"version"+str(version),Errors=Errors)

In [ ]:
# plotting forecast(i) against X_uniform(i+1), check if I have found the error correctly
# Ndum=6500
# plt.plot(time_uniform[1:Ndum+1]/24/3600/365,forecast[:Ndum,0],label='Forecast',color='b',linestyle='None',marker='o',markersize=2)
# plt.plot(time_uniform[1:Ndum+1]/24/3600/365,X_uniform[1:Ndum+1,0],label='X',color='r',linestyle='None',marker='o',markersize=2)
# plt.legend()

In [ ]:
# # find the array containing the error
# error=forecast[:-1,:]-X_uniform[1:,:]
# plt.plot(error,label='Error',color='g',linestyle='None',marker='o',markersize=2)

In [ ]:
# find the mean and std of the error along the columns
mean_error=np.mean(Errors,axis=0)

cov_matrix = np.cov(Errors-mean_error, rowvar=False)
diag_cov_matrix=np.diag(cov_matrix)
# Create a diagonal matrix with the diagonal elements of the covariance matrix
print(diag_cov_matrix)
print(mean_error)